In [25]:
# import packages
import pandas as pd
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import chromedriver_binary
import random
from datetime import timedelta, date
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
from selenium.common.exceptions import NoSuchElementException

# login and passaword
f=open("account.txt","r")
lines=f.readlines()
username=lines[0]
password=lines[1]
site = lines[2]
site_login = lines[3]
f.close()

In [43]:
driver = webdriver.Edge() 
driver.get(site)


def login():
  try:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'usuario')))       
    element.send_keys(username)
    element = driver.find_element(By.ID, "password")
    element.send_keys(password)
    element = driver.find_element_by_id('Entrar')
    element.click()
    print("Login successfully")
  except Exception as e:
    print("Login failed")
  try:
    #acess the search of the taxs
    element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
    'body > div.bg > div > div.bg.ng-scope > div.ng-scope > header > nav > div > ul > li.menu-principal.menu-dropdown')))
    element.click()
    element = WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.CSS_SELECTOR, 
    'body > div > div > div.bg.ng-scope > div.ng-scope > header > nav > div > ul > li.menu-principal.menu-dropdown.open > ul > div > li:nth-child(1) > ul > li:nth-child(1) > a')))
    element.click()
    element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.ID,'abaBuscaAvancada')))
    element.click()
    print("Acess Menu successfully")
  except Exception as e:
    print("Acess Menu failed")

login() 

C:\Users\Filipe\AppData\Local\Temp\ipykernel_10900\449561914.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id('Entrar')


Login failed
Acess Menu successfully


In [11]:
df_april = pd.read_excel("ResultadoConsultaInfracoes_abril.xlsx", skiprows=5)
df_april.columns = df_april.iloc[0]
df_april = df_april[1:]
df_test = df_april['N. Auto'].head(100)


c:\Users\Filipe\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
sne_list = []
adress_list = []
date_hour_list = []
date_list = []
hour_list = []
code_list = []
reg_list = []
# jud_list = []
origin_list = []

def reg_():
    reg = [reg.split(': ', 1)[1] for reg in reg_list]
    return(reg)

def adress_():
    adress = [adress.split(': ', 1)[1] for adress in adress_list]
    return(adress)

def code_():
    codes = [code.replace(' - ', '') for code in code_list]
    code = [code.split(': ', 1)[1] for code in codes]
    return(code)

def date_():
    date_hour = [date_hour.split(': ', 1)[1] for date_hour in date_hour_list]
    date = [date.split(' ', 1)[0] for date in date_hour]
    return(date)

def hour_():
    date_hour = [date_hour.split(': ', 1)[1] for date_hour in date_hour_list]
    hour = [hour.split(' ', 1)[1] for hour in date_hour]
    return(hour)



def souce_finn(auto):
    try:
        print("Start Search")
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'codigo')))
        element.clear()
        element.send_keys(str(auto))
        element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'btnBuscaAvancada')))
        time.sleep(0.75)
        element.click()
        element = WebDriverWait(driver, 12).until(EC.invisibility_of_element((By.CLASS_NAME, 'loading-root')))
        table_id = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#vm\.id > tbody')))

        print("Information SEI:")
        rows = table_id.find_elements(By.TAG_NAME, "tr") 
        for row in rows:
            col_11 = row.find_elements(By.TAG_NAME, "td")[11]
            print(col_11.text)
            sne_list.append(col_11.text)

        print("Accessing the info box")
        element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#vm\.id > tbody > tr > td:nth-child(2) > a')))
        time.sleep(0.75)
        element.click()
        element = WebDriverWait(driver, 12).until(EC.invisibility_of_element((By.CLASS_NAME, 'loading-root')))

        # get adress
        adress = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[5]/div/div[1]')[0].text
        if adress == '':
            adress = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[5]/div/div[2]')[0].text
        if adress == '':
            adress = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[2]')[0].text
        print(adress)
        adress_list.append(adress) 
        
        # get code 
        code = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[1]')[0].text
        if code == '':
            code =  driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[7]/div/div[1]')[0].text
        print(code)
        code_list.append(code)
        
        # get reg
        reg = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[7]')[0].text
        if "Matrícula" not in reg:
            reg = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[7]/div/div[7]')[0].text
        print(reg)
        reg_list.append(reg)

        # get  date and hours
        date_hour = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[5]/div/div[5]')[0].text
        if date_hour == '':
            date_hour = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[5]')[0].text
        print(date_hour)
        date_hour_list.append(date_hour)
               
        driver.find_element_by_css_selector("#btnFecharDetalhe").click()
    except Exception as e:
      print(e)
      print("ERROR")

In [ ]:
for i in df_test:
    if i[:2] == 'TR':
        origin = "Radar"
    else:
        origin = "DOFT"   
    origin_list.append(origin)   
    print("Auto:", i)
    souce_finn(i)

reg_list = reg_()
adress_list = adress_()
code_list = code_()
date_list = date_()
hour_list = hour_()